<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb tenacity

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog
import os

os.environ["OPENAI_API_KEY"] = ""

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
········
OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [5]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

wandb: Currently logged in as: abdalrahmenyousif (pepo). Use `wandb login --relogin` to force relogin


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [7]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [8]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

"Why am I unable to save my project in Weights and Biases?"

"How can I track my team's progress using the W&B platform?"

"Why am I not seeing any visualizations or charts in my Weights & Biases project?"

"How can I troubleshoot and fix a connection issue with my W&B app?"

How can I track the performance of different experiments in W&B?

# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [9]:
# Test if examples.txt is present, download if not
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

In [10]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")  
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "my logging doesn't seem to include errors when the training crashes, how do I change the logging level
for wandb logging?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [11]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: I am trying to
integrate gradio block in wandb. Here are my last three lines of code run = wandb.init(project=prms.WANDB_PROJECT, 
entity=prms.WANDB_ENTITY, job_type="gradio") block.launch(share=True) block.integrate(wandb=wandb) gradio app is   
launched and working, wandb run as well, but there is not logged block in wandb I have a question about tables. Is 
there a limitation in the number of rows that can be uploaded? how can i get all the versions of an aritfact of a  
particular type? Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [12]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

I am trying to integrate gradio block in wandb. The gradio app and wandb run seem to be working fine, but the block
is not being logged in wandb. Can you please help me understand why this might be happening?

 1 How can I successfully log a gradio block in wandb? I've followed the integration steps but the block is not    
   showing up in my wandb run.                                                                                     
 2 Is there a maximum limit on the number of rows that can be uploaded in a table in wandb? I have a large dataset 
   and want to ensure all rows can be accurately logged.                                                           
 3 Can you please guide me on how to retrieve all versions of an artifact of a specific type in wandb? I need to   
   access all the versions for analysis purposes.

I am having an issue with integrating the Gradio block in Wandb. I have initialized the run and launched the Gradio
app, but the block is not being logged in Wandb. Can you help me troubleshoot this issue?

"I am trying to integrate the gradio block in W&B, but I'm not seeing the block logged in W&B. Can you help me     
troubleshoot this issue?"

How can I resolve the issue of my gradio block not being logged in WandB despite launching the gradio app and      
initializing the WandB run?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [13]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

In [14]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

In [15]:
documents[0]

(PosixPath('lightning.md'),
 'import Tabs from \'@theme/Tabs\';\nimport TabItem from \'@theme/TabItem\';\n\n# PyTorch Lightning\n\n[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://wandb.me/lightning)\n\nPyTorch Lightning provides a lightweight wrapper for organizing your PyTorch code and easily adding advanced features such as distributed training and 16-bit precision. W&B provides a lightweight wrapper for logging your ML experiments. But you don\'t need to combine the two yourself: Weights & Biases is incorporated directly into the PyTorch Lightning library via the [**`WandbLogger`**](https://pytorch-lightning.readthedocs.io/en/stable/extensions/generated/pytorch\\_lightning.loggers.WandbLogger.html#pytorch\\_lightning.loggers.WandbLogger).\n\n## ⚡ Get going lightning-fast with just two lines.\n\n```python\nfrom pytorch_lightning.loggers import WandbLogger\nfrom pytorch_lightning import Trainer\n\nwandb_logger = WandbLogger()\ntrainer = Trainer(log

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [16]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[4179, 365, 1206, 2596, 2940, 537, 956, 803, 1644, 2529, 2093]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [17]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [18]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [19]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: def init(self, *args, **kwarg):                
self.save_hyperparameters()                                                                                        

                                                                                                                   
                                                                                                                   
 ### Log additional config parameters                                                                              
                                                                                                                   
 ```python                                                                                                         
 # add one parameter                                                                                               
 wandb_logger.experiment.config["key"] = value                                                                     
                                                                                                                   
 # add multiple parameters                                                                                         
 wandb_logger.experiment.config.update({key1: val1, key2: val2})                                                   
                                                                                                                   
 # use directly wandb module                                                                                       
 wandb.config["key"] = value                                                                                       
 wandb.config.update()                                                                                             
                                                                                                                   

                               Log gradients, parameter histogram and model topology                               

You can pass your model object to wandblogger.watch() to monitor your models's gradients and parameters as you     
train. See the PyTorch Lightning ]8;id=556687;https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.loggers.WandbLogger.html?highlight=wandblogger\WandbLogger]8;;\]8;id=891114;https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.loggers.WandbLogger.html?highlight=wandblogger\ documentation]8;;\ for a full description                                  

                                                    Log metrics                                                    

You can log your metrics to W&B when using the WandbLogger by calling self.log('my_metric_name', metric_vale)      
within your LightningModule, such as in your training_step or __ validation_step methods.                          

The code snippet below shows how to define your LightningModule to log your metrics and your LightningModule       
hyperparameters. In this example we will use the ]8;id=868280;https://github.com/PyTorchLightning/metrics\torchmetrics]8;;\ library to calculate our metrics                     

                                                                                                                   
 import torch                                                                                                      
 from torch.nn import Linear, CrossEntropyLoss, functional as F                                                    
 from torch.optim import Adam                                                                                      
 from torchmetrics.functional import accuracy                                                                      
 from pytorch

Let's generate 3 possible questions:

In [20]:
generate_and_print(system_prompt, generation_prompt, n=3)

Support question:                                                                                                  

How can I log additional configuration parameters using W&B?

I see that you are using W&B with PyTorch Lightning. How can I assist you with that?

Support Question: How can I add additional configuration parameters when using the WandbLogger in W&B?             

Documentation excerpt:                                                                                             

                                                                                                                   
 # add one parameter                                                                                               
 wandb_logger.experiment.config["key"] = value                                                                     
                                                                                                                   
 # add multiple parameters                                                                                         
 wandb_logger.experiment.config.update({key1: val1, key2: val2})                                                   
                                                                                                                   
 # use directly wandb module                                                                                       
 wandb.config["key"] = value                                                                                       
 wandb.config.update()                                                                                             
                                                                                                                   

Answer: You can add additional configuration parameters when using the WandbLogger in W&B by using the methods     
described in the documentation. You can either add a single parameter using wandb_logger.experiment.config["key"] =
value, add multiple parameters using wandb_logger.experiment.config.update({key1: val1, key2: val2}), or utilize   
the wandb.config module directly with wandb.config["key"] = value or wandb.config.update().

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [ ]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"

In [21]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [22]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [23]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [24]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [44]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [37]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
I have a question about exporting CSV files from a web panel. Whenever I do this, I always get two extra columns   
for MAX and MIN values, even if I only have one data curve. Does anyone know how to solve this issue? Hi How can I 
move a column in a table to the right? I have a question about sweeps. How can you constrain relationship between  
parameters. For example, I now that if num_layers * hidden_dim is large, I'll run out of GPU memory. So, also I    
would like to explore some hyperparameter space, there are some combination I know will fail. optuna as a way to do
that: you can throw an special exception to cancel a run during a sweep, so that it is not recorded. Is there      
something similar in W&B, or another way of pruning unwanted combination of hyperparameters?                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
def init(self, *args, **kwarg): self.save_hyperparameters()                                                        

                                                                                                                   
                                                                                                                   
 ### Log additional config parameters                                                                              
                                                                                                                   
 ```python                                                                                                         
 # add one parameter                                                                                               
 wandb_logger.experiment.config["key"] = value                                                                     
                                                                                                                   
 # add multiple parameters                                                                                         
 wandb_logger.experiment.config.update({key1: val1, key2: val2})                                                   
                                                                                                                   
 # use directly wandb module                                                                                       
 wandb.config["key"] = value                                                                                       
 wandb.config.update()                                                                                             
                                                                                                                   

                               Log gradients, parameter histogram and model topology                               

You can pass your model object to wandblogger.watch() to monitor your models's gradients and parameters as you     
train. See the PyTorch Lightning ]8;id=966117;https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.loggers.WandbLogger.html?highlight=wandblogger\WandbLogger]8;;\]8;id=730897;https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.loggers.WandbLogger.html?highlight=wandblogger\ documentation]8;;\ for a full descriptio

In [27]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [29]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [38]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)


 Where do I find the wandb run id?
I
I am looking to finetune LLAMA on my own dataset using OpenAI, can you give me examples on how to do this?


In [41]:
generations[0]

'CONTEXT: \nThe user is new to Weights & Biases and wants to know how to install the `wandb` library and login to their account to start using it.\n\nQUESTION: \nI just signed up for Weights & Biases. Can you tell me how to install the `wandb` library and login to my account?\n\nANSWER: \nTo install the `wandb` library and login to your Weights & Biases account, you can follow these steps:\n\n1. Open your terminal or command prompt.\n2. Run the command `pip install wandb` to install the `wandb` library.\n3. Once the installation is complete, run the command `wandb login` to log in to your account.\n4. A browser window will open, prompting you to sign in to your Weights & Biases account.\n5. After signing in, you will be redirected to the "Authorize" page.\n6. On the "Authorize" page, you will find your API key. This key will be used to authenticate your training script with your Weights & Biases account.\n\nPlease note that if you are using Weights & Biases for the first time, you migh

In [39]:
parse_generation(generations[0])

('\nThe user is new to Weights & Biases and wants to know how to install the `wandb` library and login to their account to start using it.\n',
 '\nI just signed up for Weights & Biases. Can you tell me how to install the `wandb` library and login to my account?\n',
 '\nTo install the `wandb` library and login to your Weights & Biases account, you can follow these steps:\n\n1. Open your terminal or command prompt.\n2. Run the command `pip install wandb` to install the `wandb` library.\n3. Once the installation is complete, run the command `wandb login` to log in to your account.\n4. A browser window will open, prompting you to sign in to your Weights & Biases account.\n5. After signing in, you will be redirected to the "Authorize" page.\n6. On the "Authorize" page, you will find your API key. This key will be used to authenticate your training script with your Weights & Biases account.\n\nPlease note that if you are using Weights & Biases for the first time, you might want to check out 

In [45]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

<Artifact generated_examples>

In [46]:
df.head()

,context,question,answer
0,The user is working on a machine learning proj...,I have trained a model using my MNIST dataset ...,"To load a checkpoint in W&B, you can use the `..."
1,A user wants to log images during their model ...,How can I log images with captions using Weigh...,To log images with captions using Weights & Bi...
2,\nThe user is trying to train a model using We...,\nHow can I log images with captions in W&B du...,\nTo log images with captions in W&B during th...
3,The user is working on a machine learning proj...,How can I save and load checkpoints in Weights...,To save and load checkpoints in Weights & Bias...
4,The user is using Weights & Biases (W&B) to tr...,"How can I log media types like Audio, Molecule...","To log media types like Audio, Molecules, Poin..."


In [47]:
wandb.finish()

usage/completion_tokens,▁▂▂▅▅▅▆▅▄▄▄▆▅▆▄▆▅▄▄▅▅▅▆▄█▃
usage/elapsed_time,▁▂▃▆█▄▇▆▅▅▄▇▅▆▅▆▅▅▅▅▅▆▆▄█▃
usage/prompt_tokens,▁▂▄▆▅▆▅▆▅▆▅▅▆▇▅▅▅▅▆▆█▆▆▆▇▆
usage/total_tokens,▁▂▃▆▅▆▆▅▅▅▅▆▆▇▅▆▅▅▅▆▇▆▆▅█▄
usage/completion_tokens,703
usage/elapsed_time,6.61589
usage/prompt_tokens,970
usage/total_tokens,1673
